In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential,get_bearer_token_provider
from autogen_agentchat.agents import AssistantAgent,BaseChatAgent
from autogen_core.models import UserMessage
from pydantic import BaseModel
from autogen_core.tools import FunctionTool
from autogen_agentchat.ui import Console


class model_output(BaseModel):
    question: str
    answer: str


# Create the token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-08-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider=token_provider
)


# return type for the below method is CreateResult
# using default method to create a message
result = await az_model_client.create([UserMessage(content="What is the capital of France?", source="user")],json_output= model_output)
print(result.content)
#await az_model_client.close() 


{"question":"What is the capital of France?","answer":"The capital of France is Paris."}


In [3]:
# mock tool
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


web_search_function_tool = FunctionTool(web_search, description="Find information on the web")


def add_numbers(num1 : int, num2:int)-> int:
    """ This tool will be used to add two numbers
         input is num1 and num2 of integer type
         output  is integer type
    """
    return num1+num2


tool= FunctionTool(add_numbers,description="A tool for adding number")


#Using AssitantAgent with Azure OpenAI Client

agent= AssistantAgent(
    name='Assistant', description='you are a great assistant',
    model_client=az_model_client,
    system_message='You are a really helpful assistant who help on the task given.',
    tools=[web_search_function_tool]#,
   # output_content_type=model_output   --> If this there then we are getting error with tool output
   #, output_content_type=model_output  # This is used to specify the output type of the model, which is a pydantic model in this case.
  # ,reflect_on_tool_use=False
)


output = await agent.run(
    task="What is the capital of France?")

print(type(output))  # This will print the type of the output, which should be TaskResult
print(output.messages[0].content)  # This will print the content of the message
print(output.messages[-1].content)  # This will print the JSON representation of the model_output

<class 'autogen_agentchat.base._task.TaskResult'>
What is the capital of France?
The capital of France is Paris.


In [4]:
await agent.run()
#This will return 

TaskResult(messages=[TextMessage(id='957fe69c-ba1f-4e47-9d3c-a7c28d2702cd', source='Assistant', models_usage=RequestUsage(prompt_tokens=81, completion_tokens=9), metadata={}, created_at=datetime.datetime(2025, 7, 19, 2, 57, 12, 965505, tzinfo=datetime.timezone.utc), content='The capital of France is Paris.', type='TextMessage')], stop_reason=None)

In [5]:
await Console(
        agent.run_stream(task="Find information on AutoGen"),
        output_stats=True,  # Enable stats printing.
    )

---------- TextMessage (user) ----------
Find information on AutoGen
---------- ToolCallRequestEvent (Assistant) ----------
[FunctionCall(id='call_SSaqgMCzIHHoWak50UTbbe6e', arguments='{"query":"AutoGen"}', name='web_search')]
[Prompt tokens: 102, Completion tokens: 16]
---------- ToolCallExecutionEvent (Assistant) ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', name='web_search', call_id='call_SSaqgMCzIHHoWak50UTbbe6e', is_error=False)]
---------- ToolCallSummaryMessage (Assistant) ----------
AutoGen is a programming framework for building multi-agent applications.
---------- Summary ----------
Number of messages: 4
Finish reason: None
Total prompt tokens: 102
Total completion tokens: 16
Duration: 1.79 seconds


TaskResult(messages=[TextMessage(id='7a219ef8-eba8-4a80-a8e7-7cd378011c3d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 2, 57, 22, 988122, tzinfo=datetime.timezone.utc), content='Find information on AutoGen', type='TextMessage'), ToolCallRequestEvent(id='d0442c64-4a83-434d-be2f-d6114019655f', source='Assistant', models_usage=RequestUsage(prompt_tokens=102, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 7, 19, 2, 57, 24, 771467, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_SSaqgMCzIHHoWak50UTbbe6e', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='e399b621-f83c-480f-bf86-31a9bc4b4efb', source='Assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 2, 57, 24, 776730, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.'